# 목차
- [Pandas로 데이터 읽어오기](#Pandas로-데이터-읽어오기)
- [Pandas 기초 (1)](#Pandas_기초_(1))
- [Pandas 이용한 CCTV와 인구 현황 파악](#Pandas_이용한_CCTV와_인구_현황_파악)
- [Pandas 기초 (2)](#Pandas_기초_(2))
- [CCTV 데이터와 인구 데이터 병합하고 분석하기](#CCTV_데이터와_인구_데이터_병합하고_분석하기)
- [Matplotlib 기초](#Matplotlib_기초)
- [CCTV와 인구현황 그래프로 분석하기](#CCTV와_인구현황_그래프로_분석하기)
- [조금 더 설득력 있는 자료 만들기](#조금_더_설득력_있는_자료_만들기)

In [1]:
import numpy as np
import pandas as pd

# 데이터 정리하기

In [2]:
crime_anal_police = pd.read_csv("../data/02. crime_in_Seoul.csv", encoding="euc-kr")
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,"1,395",477,"1,355","1,170"
1,종로서,3,3,6,5,115,98,"1,070",413,"1,278","1,070"
2,남대문서,1,0,6,4,65,46,"1,153",382,869,794
3,서대문서,2,2,5,4,154,124,"1,812",738,"2,056","1,711"
4,혜화서,3,2,5,4,96,63,"1,114",424,"1,015",861


#### 빈 데이터가 없는지 등을 확인

In [3]:
crime_anal_police.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   관서명     31 non-null     object
 1   살인 발생   31 non-null     int64 
 2   살인 검거   31 non-null     int64 
 3   강도 발생   31 non-null     int64 
 4   강도 검거   31 non-null     int64 
 5   강간 발생   31 non-null     int64 
 6   강간 검거   31 non-null     int64 
 7   절도 발생   31 non-null     object
 8   절도 검거   31 non-null     object
 9   폭력 발생   31 non-null     object
 10  폭력 검거   31 non-null     object
dtypes: int64(6), object(5)
memory usage: 2.8+ KB


In [4]:
len(crime_anal_police['관서명'].unique())

31

# Google Maps를 활용하여 경찰서에 관련된 정보를 가져오기

In [5]:
import googlemaps

In [6]:
%load_ext dotenv
%dotenv
import os
gmap_key = os.environ.get("GOOGLE_MAP_KEY")
gmaps = googlemaps.Client(key=gmap_key)

In [7]:
gmaps.geocode('여수돌산식당', language='ko')

[{'address_components': [{'long_name': '８３１',
    'short_name': '８３１',
    'types': ['premise']},
   {'long_name': '중앙동',
    'short_name': '중앙동',
    'types': ['political', 'sublocality', 'sublocality_level_2']},
   {'long_name': '여수시',
    'short_name': '여수시',
    'types': ['locality', 'political']},
   {'long_name': '전라남도',
    'short_name': '전라남도',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': '대한민국',
    'short_name': 'KR',
    'types': ['country', 'political']},
   {'long_name': '550-060',
    'short_name': '550-060',
    'types': ['postal_code']}],
  'formatted_address': '대한민국 전라남도 여수시 중앙동 831',
  'geometry': {'location': {'lat': 34.7395314, 'lng': 127.7347758},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 34.74088038029149,
     'lng': 127.7361247802915},
    'southwest': {'lat': 34.73818241970849, 'lng': 127.7334268197085}}},
  'place_id': 'ChIJyQ2l-NnYbTURc97KmBCbMgM',
  'types': ['establishment', 'food', 'point_of_interes

# 범죄 데이터에 주소, 위도, 경도 정보 추가하기

In [8]:
station_name = [f"서울{str(name[:-1])}경찰서" for name in crime_anal_police['관서명']]
crime_anal_police["관서명"] = station_name
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,서울중부경찰서,2,2,3,2,105,65,"1,395",477,"1,355","1,170"
1,서울종로경찰서,3,3,6,5,115,98,"1,070",413,"1,278","1,070"
2,서울남대문경찰서,1,0,6,4,65,46,"1,153",382,869,794
3,서울서대문경찰서,2,2,5,4,154,124,"1,812",738,"2,056","1,711"
4,서울혜화경찰서,3,2,5,4,96,63,"1,114",424,"1,015",861


In [9]:
def get_cord(name):
    print(gmaps.geocode(name, language='ko')[0].get('formatted_address').split()[2])
    print(list(filter(lambda x: 'sublocality_level_1' in x.get("types"), gmaps.geocode(name, language='ko')[0].get('address_components')))[0].get('long_name'))

get_cord("서울중부경찰서")

중구
중구


In [10]:
crime_anal_police["구별"] = crime_anal_police["관서명"].apply(lambda name: list(filter(lambda x: 'sublocality_level_1' in x.get("types"), gmaps.geocode(name, language='ko')[0].get('address_components')))[0].get('long_name'))
crime_anal_police

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,서울중부경찰서,2,2,3,2,105,65,"1,395",477,"1,355","1,170",중구
1,서울종로경찰서,3,3,6,5,115,98,"1,070",413,"1,278","1,070",종로구
2,서울남대문경찰서,1,0,6,4,65,46,"1,153",382,869,794,중구
3,서울서대문경찰서,2,2,5,4,154,124,"1,812",738,"2,056","1,711",서대문구
4,서울혜화경찰서,3,2,5,4,96,63,"1,114",424,"1,015",861,종로구
5,서울용산경찰서,5,5,14,14,194,173,"1,557",587,"2,050","1,704",용산구
6,서울성북경찰서,2,2,2,1,86,71,953,409,"1,194","1,015",성북구
7,서울동대문경찰서,5,5,13,13,173,146,"1,981",814,"2,548","2,227",동대문구
8,서울마포경찰서,8,8,14,10,294,247,"2,555",813,"2,983","2,519",마포구
9,서울영등포경찰서,14,12,22,20,295,183,"2,964",978,"3,572","2,961",영등포구


In [11]:
crime_anal_police["위도"] = crime_anal_police["관서명"].apply(lambda name: gmaps.geocode(name, language='ko')[0].get('geometry')['location']['lat']) # lat
crime_anal_police

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별,위도
0,서울중부경찰서,2,2,3,2,105,65,"1,395",477,"1,355","1,170",중구,37.563646
1,서울종로경찰서,3,3,6,5,115,98,"1,070",413,"1,278","1,070",종로구,37.575548
2,서울남대문경찰서,1,0,6,4,65,46,"1,153",382,869,794,중구,37.554758
3,서울서대문경찰서,2,2,5,4,154,124,"1,812",738,"2,056","1,711",서대문구,37.564744
4,서울혜화경찰서,3,2,5,4,96,63,"1,114",424,"1,015",861,종로구,37.571853
5,서울용산경찰서,5,5,14,14,194,173,"1,557",587,"2,050","1,704",용산구,37.538710
6,서울성북경찰서,2,2,2,1,86,71,953,409,"1,194","1,015",성북구,37.589778
7,서울동대문경찰서,5,5,13,13,173,146,"1,981",814,"2,548","2,227",동대문구,37.585061
8,서울마포경찰서,8,8,14,10,294,247,"2,555",813,"2,983","2,519",마포구,37.550814
9,서울영등포경찰서,14,12,22,20,295,183,"2,964",978,"3,572","2,961",영등포구,37.515318


In [12]:
crime_anal_police["경도"] = crime_anal_police["관서명"].apply(lambda name: gmaps.geocode(name, language='ko')[0].get('geometry')['location']['lng']) # lng
crime_anal_police

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별,위도,경도
0,서울중부경찰서,2,2,3,2,105,65,"1,395",477,"1,355","1,170",중구,37.563646,126.989580
1,서울종로경찰서,3,3,6,5,115,98,"1,070",413,"1,278","1,070",종로구,37.575548,126.984747
2,서울남대문경찰서,1,0,6,4,65,46,"1,153",382,869,794,중구,37.554758,126.973498
3,서울서대문경찰서,2,2,5,4,154,124,"1,812",738,"2,056","1,711",서대문구,37.564744,126.966770
4,서울혜화경찰서,3,2,5,4,96,63,"1,114",424,"1,015",861,종로구,37.571853,126.998914
5,서울용산경찰서,5,5,14,14,194,173,"1,557",587,"2,050","1,704",용산구,37.538710,126.965918
6,서울성북경찰서,2,2,2,1,86,71,953,409,"1,194","1,015",성북구,37.589778,127.016589
7,서울동대문경찰서,5,5,13,13,173,146,"1,981",814,"2,548","2,227",동대문구,37.585061,127.045768
8,서울마포경찰서,8,8,14,10,294,247,"2,555",813,"2,983","2,519",마포구,37.550814,126.954028
9,서울영등포경찰서,14,12,22,20,295,183,"2,964",978,"3,572","2,961",영등포구,37.515318,126.905728


In [13]:
crime_anal_police.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   관서명     31 non-null     object 
 1   살인 발생   31 non-null     int64  
 2   살인 검거   31 non-null     int64  
 3   강도 발생   31 non-null     int64  
 4   강도 검거   31 non-null     int64  
 5   강간 발생   31 non-null     int64  
 6   강간 검거   31 non-null     int64  
 7   절도 발생   31 non-null     object 
 8   절도 검거   31 non-null     object 
 9   폭력 발생   31 non-null     object 
 10  폭력 검거   31 non-null     object 
 11  구별      31 non-null     object 
 12  위도      31 non-null     float64
 13  경도      31 non-null     float64
dtypes: float64(2), int64(6), object(6)
memory usage: 3.5+ KB


In [14]:
crime_anal_police["절도 검거"] = crime_anal_police["절도 검거"].apply(lambda x: x.replace(',','')).astype(int)
crime_anal_police["절도 발생"] = crime_anal_police["절도 발생"].apply(lambda x: x.replace(',','')).astype(int)
crime_anal_police["폭력 발생"] = crime_anal_police["폭력 발생"].apply(lambda x: x.replace(',','')).astype(int)
crime_anal_police["폭력 검거"] = crime_anal_police["폭력 검거"].apply(lambda x: x.replace(',','')).astype(int)
crime_anal_police.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   관서명     31 non-null     object 
 1   살인 발생   31 non-null     int64  
 2   살인 검거   31 non-null     int64  
 3   강도 발생   31 non-null     int64  
 4   강도 검거   31 non-null     int64  
 5   강간 발생   31 non-null     int64  
 6   강간 검거   31 non-null     int64  
 7   절도 발생   31 non-null     int64  
 8   절도 검거   31 non-null     int64  
 9   폭력 발생   31 non-null     int64  
 10  폭력 검거   31 non-null     int64  
 11  구별      31 non-null     object 
 12  위도      31 non-null     float64
 13  경도      31 non-null     float64
dtypes: float64(2), int64(10), object(2)
memory usage: 3.5+ KB


#### 정리한 데이터 저장

In [15]:
crime_anal_police.to_csv('../data/02. crime_in_Seoul_include_gu_name.csv', sep=',', encoding='utf-8')

# pivot_table을 활용해서 데이터 정리하기

In [16]:
crime_anal_raw = pd.read_csv('../data/02. crime_in_Seoul_include_gu_name.csv', encoding='utf-8')
crime_anal_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  31 non-null     int64  
 1   관서명         31 non-null     object 
 2   살인 발생       31 non-null     int64  
 3   살인 검거       31 non-null     int64  
 4   강도 발생       31 non-null     int64  
 5   강도 검거       31 non-null     int64  
 6   강간 발생       31 non-null     int64  
 7   강간 검거       31 non-null     int64  
 8   절도 발생       31 non-null     int64  
 9   절도 검거       31 non-null     int64  
 10  폭력 발생       31 non-null     int64  
 11  폭력 검거       31 non-null     int64  
 12  구별          31 non-null     object 
 13  위도          31 non-null     float64
 14  경도          31 non-null     float64
dtypes: float64(2), int64(11), object(2)
memory usage: 3.8+ KB


In [18]:
crime_anal_raw = pd.read_csv('../data/02. crime_in_Seoul_include_gu_name.csv', encoding='utf-8', index_col="구별", usecols=["구별", "관서명", "강간 검거", "강간 발생", "강도 검거", "강도 발생", "살인 검거", "살인 발생", "절도 발생", "절도 검거", "폭력 발생", "폭력 검거"])
crime_anal_raw.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
구별,,,,,,,,,,,
중구,서울중부경찰서,2,2,3,2,105,65,1395,477,1355,1170
종로구,서울종로경찰서,3,3,6,5,115,98,1070,413,1278,1070
중구,서울남대문경찰서,1,0,6,4,65,46,1153,382,869,794
서대문구,서울서대문경찰서,2,2,5,4,154,124,1812,738,2056,1711
종로구,서울혜화경찰서,3,2,5,4,96,63,1114,424,1015,861


In [19]:
crime_anal = pd.pivot_table(crime_anal_raw, index='구별', aggfunc=np.sum)
crime_anal.head()

,강간 검거,강간 발생,강도 검거,강도 발생,살인 검거,살인 발생,절도 검거,절도 발생,폭력 검거,폭력 발생
구별,,,,,,,,,,
강남구,349,449,18,21,10,13,1650,3850,3705,4284
강동구,123,156,8,6,3,4,789,2366,2248,2712
강북구,126,153,13,14,8,7,618,1434,2348,2649
강서구,191,262,13,13,8,7,1260,2096,2718,3207
관악구,221,320,14,12,8,9,827,2706,2642,3298


In [20]:
crime_anal['강간검거율'] = crime_anal['강간 검거']/crime_anal['강간 발생']*100
crime_anal['강도검거율'] = crime_anal['강도 검거']/crime_anal['강도 발생']*100
crime_anal['살인검거율'] = crime_anal['살인 검거']/crime_anal['살인 발생']*100
crime_anal['절도검거율'] = crime_anal['절도 검거']/crime_anal['절도 발생']*100
crime_anal['폭력검거율'] = crime_anal['폭력 검거']/crime_anal['폭력 발생']*100

del crime_anal['강간 검거']
del crime_anal['강도 검거']
del crime_anal['살인 검거']
del crime_anal['절도 검거']
del crime_anal['폭력 검거']

crime_anal.head()

,강간 발생,강도 발생,살인 발생,절도 발생,폭력 발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,156,6,4,2366,2712,78.846154,133.333333,75.000000,33.347422,82.890855
강북구,153,14,7,1434,2649,82.352941,92.857143,114.285714,43.096234,88.637222
강서구,262,13,7,2096,3207,72.900763,100.000000,114.285714,60.114504,84.752105
관악구,320,12,9,2706,3298,69.062500,116.666667,88.888889,30.561715,80.109157


In [21]:
# 100이 넘는 데이터가 존해(100%를 넘는 건 이상하므로 100으로 처리, 전년도 이월 사건이 있는 것으로 추정)
con_list = ['강간검거율', '강도검거율', '살인검거율', '절도검거율', '폭력검거율']

for column in con_list:
    crime_anal.loc[crime_anal[column] > 100, column] = 100
    
crime_anal.head()

,강간 발생,강도 발생,살인 발생,절도 발생,폭력 발생,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,156,6,4,2366,2712,78.846154,100.000000,75.000000,33.347422,82.890855
강북구,153,14,7,1434,2649,82.352941,92.857143,100.000000,43.096234,88.637222
강서구,262,13,7,2096,3207,72.900763,100.000000,100.000000,60.114504,84.752105
관악구,320,12,9,2706,3298,69.062500,100.000000,88.888889,30.561715,80.109157


#### 서울시 구별 범죄 발생 건수와 검거율 데이터 정리

In [22]:
crime_anal.rename(columns = {'강간 발생':'강간', 
                             '강도 발생':'강도', 
                             '살인 발생':'살인', 
                             '절도 발생':'절도', 
                             '폭력 발생':'폭력'}, inplace=True)
crime_anal.head()

,강간,강도,살인,절도,폭력,강간검거율,강도검거율,살인검거율,절도검거율,폭력검거율
구별,,,,,,,,,,
강남구,449,21,13,3850,4284,77.728285,85.714286,76.923077,42.857143,86.484594
강동구,156,6,4,2366,2712,78.846154,100.000000,75.000000,33.347422,82.890855
강북구,153,14,7,1434,2649,82.352941,92.857143,100.000000,43.096234,88.637222
강서구,262,13,7,2096,3207,72.900763,100.000000,100.000000,60.114504,84.752105
관악구,320,12,9,2706,3298,69.062500,100.000000,88.888889,30.561715,80.109157
